## L'objectif de ce notebook d'essai est de créer la liste des articles de UN numéro à partir du sommaire trouvable sur BIUS. 

### 0. Initialisation

In [1]:
import requests
import csv
import pandas as pd
import regex as re

In [2]:
## pour cet essai on prend 1 lien url au hasard sur BIUS
url = "https://www.biusante.parisdescartes.fr/histoire/medica/resultats/index.php?do=chapitre&cote=90156x1901x75"

### 1 : Extraire de chaque page de sommaire d'un numéro sur BIUS la page et le titre des articles

#### 1.1. Extraire le texte HTML de la page et le stocker dans un document

In [3]:
Liste_entrees = []
r = requests.get(url)
print(r.text)

<!DOCTYPE html>
<html>
<head>
<meta charset="UTF-8" />
<meta name="viewport" content="width=device-width, initial-scale=1.0, maximum-scale=1.0, user-scalable=1" />

<!-- Micro-données -->
<script type="application/ld+json">
{
  "@context" : "https://schema.org",
  "@type" : "Organization",
  "name" : "Bibliothèque interuniversitaire de santé, Paris",
  "logo": "/histoire/medica/assets/images/logo-biusante-officiel.png",
  "url" : "https://www.biusante.parisdescartes.fr",
  "sameAs" : [
    "https://www.facebook.com/biusante",
    "https://twitter.com/BIUSante",
    "https://plus.google.com/u/0/b/105189366630654239462/105189366630654239462/posts"
  ]
}
</script>

<!-- Favicon -->
<link rel="icon" href="https://u-paris.fr/wp-content/uploads/2019/04/Universite_Paris_Favicon.png" sizes="32x32">

<!-- Polices -->
	<link rel="stylesheet" href="https://fonts.googleapis.com/css?family=Open+Sans:400,700&amp;subset=latin,latin-ext" />

<!-- Feuilles de styles -->
	<link rel="stylesheet" href="ht

#### 1.2. Utiliser BeautifulSoup pour analyser l'HTML

In [4]:
from bs4 import BeautifulSoup
from lxml import html

In [5]:
numero_AMN_soup = BeautifulSoup (r.text,features="lxml")
print(numero_AMN_soup)

<!DOCTYPE html>
<html>
<head>
<meta charset="utf-8"/>
<meta content="width=device-width, initial-scale=1.0, maximum-scale=1.0, user-scalable=1" name="viewport"/>
<!-- Micro-données -->
<script type="application/ld+json">
{
  "@context" : "https://schema.org",
  "@type" : "Organization",
  "name" : "Bibliothèque interuniversitaire de santé, Paris",
  "logo": "/histoire/medica/assets/images/logo-biusante-officiel.png",
  "url" : "https://www.biusante.parisdescartes.fr",
  "sameAs" : [
    "https://www.facebook.com/biusante",
    "https://twitter.com/BIUSante",
    "https://plus.google.com/u/0/b/105189366630654239462/105189366630654239462/posts"
  ]
}
</script>
<!-- Favicon -->
<link href="https://u-paris.fr/wp-content/uploads/2019/04/Universite_Paris_Favicon.png" rel="icon" sizes="32x32"/>
<!-- Polices -->
<link href="https://fonts.googleapis.com/css?family=Open+Sans:400,700&amp;subset=latin,latin-ext" rel="stylesheet"/>
<!-- Feuilles de styles -->
<link href="https://www.biusante.parisd

##### 1.2.1. J'ai repéré que les références de chaque article se trouvent à l'intérieur de balises : div class="tableau1".  Je les extrait donc dans une var 'div_ens_tag'

In [6]:
div_ens_tag = numero_AMN_soup.find_all("div", {"class":"tableau1"})

# print(div_ens_tag)

##### 1.2.2. je voudrais ici extraire des href l'année / le numéro de l'article

In [7]:
for balise in div_ens_tag:
    l = list(balise)
    ahref = str(l[1])
    intermed = re.search(r"x([0-9]{4})x([0-9]{2})",ahref)
    intermed = str(intermed)

    #isoler année : 
    if re.search(r"[0-9]{4}",intermed) is None:
        pass
    else :
        revue_annee = re.search(r"[0-9]{4}",intermed).group(0)

    #isoler numéro : 
    if re.search(r"[0-9]{4}",intermed) is None:
        pass
    else :
        revue_numero = re.search(r"([0-9]{2})'",intermed).group(1)


### 2. Nettoyer les entrées pour ne conserver que les articles

#### 2.1.   Liste ne conservant que la page et le contenu de l'entrée

<b> NB </b> : je fais exprès de dire "entrée" et non "article" car il y a d'autres contenus que des articles

In [8]:
liste_page_entrees = []
Liste_entrees = []

for balise in div_ens_tag:
    liste_page_entrees.append(balise.text)

for entree in liste_page_entrees :
    RE_entree = re.findall(r"\t(\d{1,3})(.*)",entree)
    Liste_entrees.append(RE_entree)


#### 2.2. Pour les pages d'un nouvel article qui débute, ne conserver que le nom du nouveau

In [9]:
#le _t_ = "type"

Liste_entrees_no_doublesentrees = []

for entree_t_list in Liste_entrees :
    for entree_t_tupple in entree_t_list :
            tupple_to_list = list(entree_t_tupple)
            tupple_to_list.append(revue_annee)
            #type(revue_annee))= str
            tupple_to_list.append(revue_numero)
            Liste_entrees_no_doublesentrees.append(tupple_to_list)

### Etape compliquée ar il n'y a pas de symbole différent entre
### La séparation entre 2 articles et entre 2 sections d'un même article...

###print(Liste_entrees_no_doublesentrees)

#### 2.3 Retirer les sections qui ne nous intéressent pas

In [10]:
Liste_articles = []

for entree in Liste_entrees_no_doublesentrees :

    #Bulletins officiels 
    if re.search(r"(.*)?Bulletin officiel(.*)?", entree[1]):
        pass 

    #Journaux
    elif re.search(r"(.*)?((J|j)ournaux)(.*)?",entree[1]):
        pass

    #Livres recus
    elif re.search(r"(.*)?(Livres reçus)(.*)?",entree[1]):
        pass

    #Bibliographies
    elif re.search(r"(.*)?(Bibliographies?)(.*)?",entree[1]):
        pass

    #Table analytique
    elif re.search(r"(.*)?(Table (analytique)?)(.*)?",entree[1]):
        pass

    #Nécrologies
    elif re.search(r"(.*)?(N(e|é)crologie)(.*)?",entree[1]):
        pass

    else :
        Liste_articles.append(entree)
#print(Liste_articles)

In [11]:
#Liste_articles

### 3. Création d'un dataframe

In [12]:
#### 3.1 List to df

In [13]:
df = pd.DataFrame(Liste_articles)
print(df)
df.columns = ['article_page', 'article_titre','revue_annee','revue_numero']

       0                                                  1     2   3
0      5   Influence des climats et des saisons sur les ...  1901  75
1      6   Influence des climats et des saisons sur les ...  1901  75
2      7   Influence des climats et des saisons sur les ...  1901  75
3      8   Influence des climats et des saisons sur les ...  1901  75
4     10   Influence des climats et des saisons sur les ...  1901  75
..   ...                                                ...   ...  ..
148  456   Mission hydrographique de l'aviso-transport l...  1901  75
149  458   Mission hydrographique de l'aviso-transport l...  1901  75
150  462   Mission hydrographique de l'aviso-transport l...  1901  75
151  463   Mission hydrographique de l'aviso-transport l...  1901  75
152  464   À propos de l'yerba maté, par le Dr Dedet, .....  1901  75

[153 rows x 4 columns]


#### 3.2 Séparation titre de l'article // auteur

<i> 3.2.1. créer une nouvelle colonne avec nom(s)  auteur(s) (et ce qui suit) : </i>

In [14]:
df

,article_page,article_titre,revue_annee,revue_numero
0,5,Influence des climats et des saisons sur les ...,1901,75
1,6,Influence des climats et des saisons sur les ...,1901,75
2,7,Influence des climats et des saisons sur les ...,1901,75
3,8,Influence des climats et des saisons sur les ...,1901,75
4,10,Influence des climats et des saisons sur les ...,1901,75
...,...,...,...,...
148,456,Mission hydrographique de l'aviso-transport l...,1901,75
149,458,Mission hydrographique de l'aviso-transport l...,1901,75
150,462,Mission hydrographique de l'aviso-transport l...,1901,75
151,463,Mission hydrographique de l'aviso-transport l...,1901,75


In [15]:
SearchAuteur = "(.*) (P|p)ar (.*)"
df['article_auteur'] = df["article_titre"].str.split(SearchAuteur).str[3]


<i> 3.2.2. retirer nom(s) auteur(s) (et ce qui suit) de article_titre</i>

In [16]:
df['article_titre'] = df.article_titre.str.replace(SearchAuteur, r"\1") 
print(df)

    article_page                                      article_titre  \
0              5   Influence des climats et des saisons sur les ...   
1              6   Influence des climats et des saisons sur les ...   
2              7   Influence des climats et des saisons sur les ...   
3              8   Influence des climats et des saisons sur les ...   
4             10   Influence des climats et des saisons sur les ...   
..           ...                                                ...   
148          456   Mission hydrographique de l'aviso-transport l...   
149          458   Mission hydrographique de l'aviso-transport l...   
150          462   Mission hydrographique de l'aviso-transport l...   
151          463   Mission hydrographique de l'aviso-transport l...   
152          464                          À propos de l'yerba maté,   

    revue_annee revue_numero  \
0          1901           75   
1          1901           75   
2          1901           75   
3          1901    

In [17]:
df.iloc[3,1]

" Influence des climats et des saisons sur les dépenses de l'organisme chez l'homme. Fixation de la ration dans ces diverses conditions."

####  3.3. séparer l'auteur du détail de l'article

In [18]:
##Séparer les noms d'auteurs du détail des articles 
SearchDetail = "(.*)\.{2,3}(.*)"
df['article_detail'] = df["article_auteur"].str.split(SearchDetail).str[2]

#retirer nom auteurs (et ce qui suit) de article_titre
for index in range(len(df.article_auteur)-1):
    if type(df.iloc[index,4]) == str:
        df.iloc[index,4] = re.sub(SearchDetail, r"\1", df.iloc[index,4])
        

# print(df)

In [19]:
## Vérification : 
print("description : ",df.iloc[3,5])
print("\n")
print("auteur : ",df.iloc[3,4])

description :   (Suite.). VII. Recherches relatives à l'influence des saisons sur les dépenses de l'organisme (1886 à 1890). Influence des saisons sur les dépenses des animaux. Expériences faites sur les cobayes. / Première expérience


auteur :  le Dr E. Maurel, .


#### 3.4. INSERER : article_id 
      modèle de l'id : NomRevue-Numero-Pagedébut

In [20]:
df['article_id'] = "AHMC-"+df["revue_numero"]+"-"+df["article_page"]
print(df['article_id'])

0        AHMC-75-5
1        AHMC-75-6
2        AHMC-75-7
3        AHMC-75-8
4       AHMC-75-10
          ...     
148    AHMC-75-456
149    AHMC-75-458
150    AHMC-75-462
151    AHMC-75-463
152    AHMC-75-464
Name: article_id, Length: 153, dtype: object


In [21]:
df.iloc[3,3]

'75'

### 4. Mise au propre pour exportation

#### 4.1. Réorganisation des colonnes suite aux divers ajouts

In [22]:
cols = df.columns.tolist()
cols = [cols[6],cols[4],cols[5],cols[1],cols[2],cols[3],cols[0]]
df=df[cols]
print(df.iloc[3,3])

 Influence des climats et des saisons sur les dépenses de l'organisme chez l'homme. Fixation de la ration dans ces diverses conditions.


#### 4.2. Retirer les white spaces en début de string 

In [23]:
for i in range(len(df)):
    for col in range (1,4):
        if type(df.iloc[i,col]) is str :
            if re.match(r"^ ",df.iloc[i,col]):
                df.iloc[i,col] = df.iloc[i,col].replace(r"(^ )",r"")

#### 4.3. Retirer les doublons d'article 

In [24]:
# df=df.drop_duplicates(subset=['article_titre'])
df=df.drop_duplicates(subset=['article_auteur','article_titre'])

#### 4.3. Vérification avant exportation du df en csv

In [25]:
df

,article_id,article_auteur,article_detail,article_titre,revue_annee,revue_numero,article_page
0,AHMC-75-5,"le Dr E. Maurel, .",(Suite.). / VII. Recherches relatives à l'inf...,Influence des climats et des saisons sur les ...,1901,75,5
1,AHMC-75-6,"le Dr E. Maurel, .",(Suite.). VII. Recherches relatives à l'influ...,Influence des climats et des saisons sur les ...,1901,75,6
11,AHMC-75-24,les divers climats et les diverses saisons,NaN,Influence des climats et des saisons sur les ...,1901,75,24
12,AHMC-75-25,"le Dr Lesueur-Florent,",,Influence des climats et des saisons sur les ...,1901,75,25
13,AHMC-75-28,"le Dr Lesueur-Florent, .",/ Observation I,"Abcès du foie et cholerrhagie,",1901,75,28
15,AHMC-75-39,"le Dr Gauran,",,"Abcès du foie et cholerrhagie, par le Dr Lesu...",1901,75,39
16,AHMC-75-50,"le Dr Gauran, .",/ Coffres à médicaments,Notes médicales recueillies pendant une campa...,1901,75,50
17,AHMC-75-52,"le Dr Lassabatie,",,Notes médicales recueillies pendant une campa...,1901,75,52
18,AHMC-75-58,"le Dr J.-A. Portengen,",,"Mobilier chirurgical, par le Dr Lassabatie, ....",1901,75,58
19,AHMC-75-60,"le Dr J.-A. Portengen, .",/ A. Dans la marine anglaise. / B. Dans la ma...,Étude sur la statistique médicale des différe...,1901,75,60


In [26]:
df.iloc[3,3]

" Influence des climats et des saisons sur les dépenses de l'organisme chez l'homme. Fixation de la ration dans ces diverses conditions. Par le Dr E. Maurel, ... (Suite.). VIII. Recherches sur l'excrétion de l'urée. Première série. Rapport de la quantité durée excrétée avec la quantité d'azotés contenus dans la ration d'entretien, telle que je l'ai adoptée par les divers climats et les diverses saisons. (A suivre.). / Abcès du foie et cholerrhagie,"